In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import datetime
import time
import numpy as np
import pandas as pd
import random
import requests
import datetime
import re
import pyautogui

아실에서 실거래가 수집
<네이버와 비교했을 때 오류를 일으키는 항목>
차이점 1 : 수집된 주소가 조금 다르다.(서울시->서울 로 표기)
차이점 2 : 아파트명의 표기가 아파트 정보확인 데이터(check aptid)와 조금씩 다르다.
(예를 들어 e-편한세상 vs e편한세상, 대치래미안하이스턴 vs 래미안대치하이스턴)
차이점 3 : 월세의 보증금이 없는 데이터가 존재한다.


In [4]:
def clickkeyword(keyword):
    
    i=1
    while True:
        city=driver.find_element(By.CSS_SELECTOR,"#wrap > form > div.area_list_wrap.group.asilScroll > div.area_list > span:nth-child({})".format(i)).text
        try:
            if city.startswith(keyword):
                driver.find_element(By.CSS_SELECTOR,"#wrap > form > div.area_list_wrap.group.asilScroll > div.area_list > span:nth-child({})".format(i)).click()
                break
        except:
            print("some error occured")
            break
        else:
            i+=1
    
    time.sleep(2)
    return None

In [5]:
def make_intcost(text): 
    #given data is like "9억만원" or "9억 3백만원"
    if "억만원" in text:
        text=text.replace(" ","")
        text=text.replace("억","")
        text=text.replace("만원","")
        text=text.replace(",","")
        return int(text)*100000000
    elif "만원" in text:
        if text.find("억")!=-1:
            bil=text[:text.find("억")]
            mil=text.replace(bil+"억","").replace("만원","").replace(",","").replace(" ","")
            return int(bil)*100000000+int(mil)*10000
        else:
            mil=text.replace("만원","").replace(",","")
            return int(mil)*10000
    
    else:
        print("cost_change error")
        return text

In [6]:
def make_intcost2(text):
    #월세가 붙어있는 자료일때 사용한다.
    #Data is like "8억/30만원" or "9억 5,000/30만원" or "5,000/30만원"
    l=[]
    idx=text.find('/')
    l.append(int(text[idx+1:-2].replace(",","")))
    text=text[:idx]+"만원"
    if text=="만원":
        l.append(0)
    else:
        l.append(make_intcost(text))
    return l

In [7]:
make_intcost2("8억 500/30만원")

[30, 805000000]

In [8]:
def makedate(year,month,day):
    #Date 형식으로 저장한다.
    #given data is like "2023","2월","9일"
    a= datetime.date(int(year),int(month[:-1]),int(day[:-1])).strftime("%y/%m/%d")
    return a

In [9]:
def purespace(spacestring):
    for i,v in enumerate(spacestring):
        if v.isalpha():
            return int(spacestring[:i])
    return int(spacestring)

In [11]:
apt_id=pd.read_csv("C:\\Users\\Taewan\\Desktop\\the greta\\check aptid.csv",encoding='utf=8')

In [12]:
def get_aptid(loc,apt):
    #아파트 코드 찾아와서 저장하기
    
    l=loc.split(" ")
    if l[0] in ['서울','부산','대구','인천','광주','대전','울산','세종']:
        l[0]=l[0]+"시"
    elif l[0] in ['경기','강원','제주']:
        l[0]=l[0]+"도"
    elif l[0]=="충북":
        l[0]='충청북도'
    elif l[0]=="충남":
        l[0]='충청남도'
    elif l[0]=="전북":
        l[0]='전라북도'
    elif l[0]=="전남":
        l[0]='전라남도'
    elif l[0]=="경북":
        l[0]='경상북도'
    else:
        l[0]='경상남도'
 
    loc=" ".join(l)

    #loc와 일치하는 주소가 하나뿐이면 아래의 과정을 시행할 필요가 없으므로 이를 반영하기
    selected_df=apt_id[(apt_id['address_place']==loc)]
    if len(selected_df['apt_id'])==1:
        return selected_df['apt_id'].iloc[0]
    
    #그렇지 않으면 정석대로 시행해야 한다.
    selected_df=apt_id[(apt_id['apt_name']==apt) & (apt_id['province']==l[0]) & (apt_id['district']==l[1]) & (apt_id['dong']==l[2]) & (apt_id['type']=="아파트")]

    if len(selected_df['apt_id'])==1:
        return selected_df['apt_id'].iloc[0]
    elif len(selected_df['apt_id'])>1:
        print("중복되는 apt id 발견")
        return None
    else:
        #대치래미안하이스턴과 같은 표기의 혼재가 있어서 처리해 주었다.(대치래미안하이스턴=래미안대치하이스턴)
        try:
            for k in range(len(apt_id)):
                testapt=apt_id['apt_name'][k]
                if len(apt)!=len(testapt):
                    continue
                breaking=True
                for letter in testapt:
                    if letter not in apt:
                        breaking=False
                        break
                if breaking==True:
                    #print(testapt)
                    return apt_id[apt_id['apt_name']==testapt]['apt_id'].iloc[0]

        except:
            return None

In [13]:

#biglst=[]
def asiltrade(keyword1,keyword2,keyword3):
    global driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url='https://asil.kr/asil/index.jsp'
    driver.get(url)

    #keyword1="서울시"
    #keyword2="강남구"
    #keyword3="개포동"

    driver.switch_to.frame(driver.find_element(By.XPATH,'//*[@id="sub1"]'))

    ### find 시/구/동 클릭 ###
    clickkeyword(keyword1)
    clickkeyword(keyword2)
    clickkeyword(keyword3)



    i=1
    while True:
        #오피스텔과 연립다세대는 필요없다.
        driver.find_element(By.CSS_SELECTOR,"#buildingO").click()
        time.sleep(0.5)
        driver.find_element(By.CSS_SELECTOR,"#buildingD").click()
        time.sleep(2)

        #단지의 기본정보를 수집한다.
        try:
            aptname=driver.find_element(By.CSS_SELECTOR,"body > div > div.apt_list_area > div > div:nth-child({}) > p.title".format(i)).text
        #해당 키워드의 단지정보가 더 이상 없다.
        except:
            break
        
        sede=driver.find_element(By.CSS_SELECTOR,'body > div > div.apt_list_area > div > div:nth-child({}) > p.info > span:nth-child(2)'.format(i)).text
        sede=int(sede.replace(",","")[:-2])
        dong=driver.find_element(By.CSS_SELECTOR,"body > div > div.apt_list_area > div > div:nth-child({}) > p.info > span:nth-child(3)".format(i)).text
        dong=int(dong[:-2])
        addr=driver.find_element(By.CSS_SELECTOR,'body > div > div.apt_list_area > div > div:nth-child({}) > p.addr'.format(i)).text
        aptid=get_aptid(addr,aptname)

        #단지들에 대해서 접근해야한다.(상세 정보확인을 위해서)
        driver.find_element(By.CSS_SELECTOR,"body > div > div.apt_list_area > div > div:nth-child({}) > p.addr".format(i)).click()
        time.sleep(2)
        
        
        #거래현황 더보기 실행
        driver.find_element(By.CSS_SELECTOR,"body > div.asilScroll > div.apt_info > div.article.apt_info_chart.mt0 > div.table_view.mt15 > a:nth-child(3)").click()
        time.sleep(2)
        driver.switch_to.default_content()
        time.sleep(0.5)
        driver.switch_to.frame(driver.find_element(By.XPATH,'//*[@id="sub2"]'))
        time.sleep(2)

        #평풀기 실행
        try:
            driver.find_element(By.CSS_SELECTOR,"body > div:nth-child(2) > a").click()
        except:
            #아파트 단지는 있으나. 조회되는 실거래가 정보가 아예 없는경우, 실거래 수집이 핵심인데, 평풀기가 불가하므로
            #닫아서 초기 화면으로 돌아간 후 continue 처리한다.
            driver.switch_to.default_content()
            time.sleep(0.2)
            driver.switch_to.frame(driver.find_element(By.XPATH,'//*[@id="sub1"]'))
            time.sleep(0.2)
            driver.find_element(By.CSS_SELECTOR,"body > div:nth-child(1) > span").click()
            time.sleep(0.2)
            print(str(i)+"th 단지 complete")
            i+=1
            continue

        x=1
        #평수별로 크롤링을 할 것이기 때문에, 해당 평에 대한 info를 클릭해주는 작업이 필요하다.
        while True:
            driver.switch_to.default_content()
            time.sleep(0.5)
            driver.switch_to.frame(driver.find_element(By.XPATH,'//*[@id="sub2"]'))
            time.sleep(2)

            #x번째 면적에 대한 실거래가 정보
            try:
                driver.find_element(By.CSS_SELECTOR,"#mCSB_2_container > ul > li:nth-child({}) > a".format(x)).click()
            #스크롤이 덜 넘어가서 더 수집이 안되는 경우 방지
            except:
                try:
                    driver.find_element(By.CSS_SELECTOR,"#mCSB_2_container > ul > li:nth-child({}) > a".format(x-1)).click()
                    time.sleep(0.2)
                    pyautogui.keyDown('right')
                    time.sleep(0.4)
                    pyautogui.keyUp('right')
                    driver.find_element(By.CSS_SELECTOR,"#mCSB_2_container > ul > li:nth-child({}) > a".format(x)).click()
                #스크롤을 해도 수집이 안되면 수집 종료
                except:
                    break

            time.sleep(2)
            driver.switch_to.frame(driver.find_element(By.XPATH,'//*[@id="ifrm"]')) 
            
            #실거래가 더보기를 클릭이 불가능할 때 까지 클릭해 준다.
            while True:
                try:
                    driver.find_element(By.CSS_SELECTOR,"#morePriceBtn > span").click()
                    #코드 속도를 좌우한다. time.sleep(0.5)로하니 회사 와이파이로는 잘 작동하지 않는다.(학교와이파이로는 잘 됨)
                    time.sleep(0.8)
                except:
                    break
            
            ##스크롤 다 내린후(더보기 버튼 클릭 후), soup 객체 이용해서 필요한 정보를 총수집한다.
            html = driver.page_source
            time.sleep(1)
            soup = BeautifulSoup(html, 'html.parser')

            allday=soup.find_all('span', attrs={'class' : 'dd'})
            time.sleep(0.02)
            alltype=soup.find_all('span', attrs = {'class' : 'b'})
            time.sleep(0.02)
            allprice=soup.find_all('td', attrs = {'class' : re.compile('^price taL crc')})
            time.sleep(0.02)
            allyearspace=soup.find_all('span', attrs={'class' : 'mm'})
            time.sleep(0.02)
            allfloor=soup.find_all('span', attrs={"class" : "aptfloor"})
            time.sleep(0.02)
           
            
            for idx in range(len(allday)):
                if idx==0:
                    year=allyearspace[idx*2].text
                else:
                    newyear=allyearspace[idx*2].text
                    if newyear=="":
                        newyear=year
                    else:
                        year=newyear
                space=allyearspace[idx*2+1].text
                day=allday[idx].text+"일"
                type=alltype[idx].text
                pr=allprice[idx].find('span',attrs={'class': "eb"}).text
                price=pr.replace(" ","")+"만원"
                floor=allfloor[idx].text
                y="20"+year.split(".")[0]
                m=year.split(".")[1]+"월"

                if "매매" in type:
                    type="매매"
                elif "전세" in type:
                    type='전세'
                else:
                    type='월세'
                
                if type=="월세":
                    l=make_intcost2(price)
                    lst=[sede,dong,addr,aptname,aptid,int(floor[:-1]),makedate(y,m,day),type,space,purespace(space),l[1],l[0],datetime.datetime.now().strftime("%y/%m/%d")]
                else:
                    lst=[sede,dong,addr,aptname,aptid,int(floor[:-1]),makedate(y,m,day),type,space,purespace(space),make_intcost(price),None,datetime.datetime.now().strftime("%y/%m/%d")]
                print(lst)
                biglst.append(lst)
            x+=1
            
        #닫아서 초기 화면으로 돌아가기
        driver.switch_to.default_content()
        time.sleep(0.2)
        driver.switch_to.frame(driver.find_element(By.XPATH,'//*[@id="sub1"]'))
        time.sleep(0.2)
        driver.find_element(By.CSS_SELECTOR,"body > div:nth-child(1) > span").click()
        time.sleep(0.2)
        print(str(i)+"th 단지 complete")
        i+=1
            
        



In [14]:
biglst=[]

In [82]:
#주의점! pyautogui를 이용해서, 다른 창을 띄우고 크롤링 동시에 불가함(면적을 오른쪽끝까지 스크롤 불가 유의)
asiltrade("서울시","강남구","개포동")

[1169, 8, '서울 강남구 개포동 12', 'SH대치1단지', None, 9, '18/07/14', '월세', '25', 25, 30620000, 7, '23/02/20']
[1169, 8, '서울 강남구 개포동 12', 'SH대치1단지', None, 11, '18/03/29', '월세', '25', 25, 38650000, 17, '23/02/20']
[1169, 8, '서울 강남구 개포동 12', 'SH대치1단지', None, 8, '17/08/31', '월세', '25', 25, 18150000, 4, '23/02/20']
[1169, 8, '서울 강남구 개포동 12', 'SH대치1단지', None, 8, '17/04/26', '월세', '25', 25, 7300000, 10, '23/02/20']
[1169, 8, '서울 강남구 개포동 12', 'SH대치1단지', None, 12, '17/02/22', '월세', '25', 25, 22530000, 12, '23/02/20']
1th 단지 complete
[2296, 31, '서울 강남구 개포동 1282', '개포래미안포레스트', 119219, 4, '23/02/13', '월세', '49', 49, 246360000, 29, '23/02/20']
[2296, 31, '서울 강남구 개포동 1282', '개포래미안포레스트', 119219, 4, '23/02/13', '월세', '49', 49, 246360000, 29, '23/02/20']
[2296, 31, '서울 강남구 개포동 1282', '개포래미안포레스트', 119219, 3, '23/02/13', '월세', '49', 49, 188160000, 58, '23/02/20']
[2296, 31, '서울 강남구 개포동 1282', '개포래미안포레스트', 119219, 2, '23/02/13', '월세', '49', 49, 283990000, 41, '23/02/20']
[2296, 31, '서울 강남구 개포동 1282', '개포래미안포레스트', 1

KeyboardInterrupt: 

In [16]:
#자동화
global keyword1
global biglst
keyword1="서울시"
keyword2="강남구"
keyword3=["청담동"]
biglst=[]
for v in keyword3:
    asiltrade(keyword1,keyword2,v)
    pddf2=pd.DataFrame(biglst)
    #동에 거래매물이 없다면..? DataFrame is empty.
    try:
        pddf2.columns=['총 세대수','동수','주소',"아파트명",'아파트번호','해당층','실거래일시','매매유형','타입면적','면적','실거래가','월세','수집일시']
        pddf2.to_csv("C:\\Users\\Taewan\\Desktop\\새 폴더\\asil{}.csv".format(keyword1+" "+keyword2+ " "+v),encoding='euc-kr')
    except:
        None
    

[496, 13, '서울 강남구 일원동 735', '가람', 860, 4, '23/02/18', '전세', '75', 75, 550000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 3, '23/02/08', '전세', '75', 75, 480000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 1, '23/01/30', '전세', '75', 75, 600000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 1, '23/01/20', '전세', '75', 75, 580000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 3, '23/01/07', '매매', '75', 75, 1675000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 5, '22/12/07', '전세', '75', 75, 650000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 2, '22/11/30', '전세', '75', 75, 800000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 2, '22/11/30', '전세', '75', 75, 840000000, None, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 2, '22/11/29', '월세', '75', 75, 380000000, 80, '23/02/23']
[496, 13, '서울 강남구 일원동 735', '가람', 860, 1, '22/11/26', '전세', '75', 75, 680000000, None, '23/02/23']
[496, 13, '

KeyboardInterrupt: 

In [17]:
pddf2=pd.DataFrame(biglst)
#동에 거래매물이 없다면..? DataFrame is empty.
try:
    pddf2.columns=['총 세대수','동수','주소',"아파트명",'아파트번호','해당층','실거래일시','매매유형','타입면적','면적','실거래가','월세','수집일시']
    pddf2.to_csv("C:\\Users\\SAMSUNG\\Desktop\\finaldata\\asil{}.csv".format(keyword1+" "+keyword2+ " "+v),encoding='euc-kr')
except:
    None